In [1]:
#importing necessary packages
import pandas as pd
import recordlinkage
import pathlib

In [2]:
#Read cleaned data into dataframe
Data_folder: str = pathlib.Path('C:\BBBrothers\BBB BETTER AND BETTER FILES\Data\merged_file.csv')
record_data = pd.read_csv(Data_folder)
record_data

C:\Users\arcoo\AppData\Local\Temp\ipykernel_29652\2676414173.py:3: DtypeWarning: Columns (2,3,4,5,11,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  record_data = pd.read_csv(Data_folder)


,ID,firm_id,company_name,condensed_name,main,legal,address_1,address_2,city,state,zip,zip_4,phone,email,url
0,1,2.0,"Able Fence, Inc.",ablefenceinc,True,True,78 Acker St E,NaN,Saint Paul,MN,55117,5507,6512224355,NaN,http://www.albinchapel.com/
1,2,5.0,Albin Chapel,albinchapel,False,False,2200 Nicollet Ave,NaN,Minneapolis,MN,55404,3301,9529149410,office@albinchapel.com,NaN
2,3,5.0,Albin Funeral Chapel Inc,albinfuneralchapelinc,False,False,6855 Rowland Rd,NaN,Eden Prairie,MN,55344,3209,6128711418,jimalbinson@gmail.com,http://www.arthurwilliamsoptical.com/
3,4,5.0,"Albin Endeavor, Inc.",albinendeavorinc,True,True,PO Box 46147,NaN,Eden Prairie,MN,55344,2847,6122700491,NaN,NaN
4,5,7.0,Albrecht Company,albrechtcompany,True,False,2024 Blackberry Ln,NaN,Wayzata,MN,55391,2006,6516334510,mail@albrechtcompany.com,http://www.ablemovers.net
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309324,327934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7016621044,NaN,NaN
309325,327935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5072084001,NaN,NaN
309326,327936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5072084150,NaN,NaN
309327,327937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6125672270,NaN,NaN


In [3]:
#Setup indexer to make candiate pairs of records within the dataset. Block candiate pair generation on records that have
#valid condensed names. A data sample of 100000 is used for computing constraints. 
indexer = recordlinkage.Index()
data_sample = record_data.head(100000)
indexer.block("condensed_name")
indexer.block("address_1")
canidate_links = indexer.index(data_sample)
print(len(data_sample), len(canidate_links))

100000 48303


In [4]:
#Setup comparision methods used to determine potential records that match. Comparision methods can either be
#Numeric, string-based, or exact matches. Specific comparrsion methods are a work in progress to determine which
#features/methods are the best for generating accurate matches. 
compare_cl = recordlinkage.Compare()
compare_cl.string("condensed_name", "condensed_name",method ="levenshtein",threshold=0.75, label="condensed_name")
compare_cl.string(
    "company_name", "company_name", method="levenshtein", threshold=0.75, label="company_name"
)
compare_cl.string("address_1", "address_1", method="levenshtein", threshold=0.85, label="address_1")
compare_cl.exact("state", "state", label="state")
compare_cl.string(
    "email", "email", method="levenshtein", threshold=0.85, label="email"
)
compare_cl.string(
    "url", "url", label = "url"
)
# Computes the matching score of each record based on the candiate links generated and the comparision criteria. 
features = compare_cl.compute(canidate_links, data_sample)

In [5]:
features.head(10)

,,condensed_name,company_name,address_1,state,email,url
29,27,1.0,1.0,0.0,0,0.0,0.484848
220,219,0.0,0.0,1.0,1,0.0,0.000000
377,376,1.0,1.0,0.0,1,0.0,0.000000
563,180,0.0,0.0,1.0,1,0.0,0.419355
566,177,0.0,0.0,1.0,1,0.0,0.600000
624,581,1.0,1.0,0.0,0,0.0,0.586207
705,704,0.0,0.0,1.0,1,0.0,0.000000
771,770,0.0,0.0,1.0,1,0.0,0.483871
793,434,0.0,0.0,1.0,1,0.0,0.226667
1016,123,0.0,0.0,1.0,1,0.0,0.387097


In [6]:
features.describe()

,condensed_name,company_name,address_1,state,email,url
count,48303.000000,48303.000000,48303.000000,48303.000000,48303.000000,48303.000000
mean,0.308718,0.304432,0.693063,0.823117,0.000455,0.257542
std,0.461969,0.460171,0.461228,0.381574,0.021337,0.243681
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
50%,0.000000,0.000000,1.000000,1.000000,0.000000,0.300000
75%,1.000000,1.000000,1.000000,1.000000,0.000000,0.484848
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
#Finds the similarity score of overall records which is based on the sum of each feature being matched.
round(features.sum(axis=1)).value_counts().sort_index(ascending=False)

5.0       22
4.0     2822
3.0    14371
2.0    27863
1.0     3225
dtype: int64

In [8]:
#Potential matches are extracted from the features based on an arbirtray score of 3.5.
matches = features[round(features.sum(axis=1)) >= 3.5]
matches

,,condensed_name,company_name,address_1,state,email,url
1466,1465,1.0,1.0,0.0,1,0.0,0.708333
1922,1920,1.0,1.0,0.0,1,0.0,0.640000
2487,2486,1.0,1.0,0.0,1,0.0,0.680000
2574,1206,1.0,1.0,0.0,1,0.0,0.551724
2670,2669,1.0,1.0,0.0,1,0.0,0.564103
...,...,...,...,...,...,...,...
97945,97944,1.0,1.0,1.0,1,0.0,0.000000
98003,98002,1.0,1.0,1.0,1,1.0,0.000000
98014,98013,1.0,1.0,1.0,1,0.0,0.000000
98084,98083,1.0,1.0,1.0,1,0.0,0.000000


In [10]:
#The pairs of potential records are given IDs to be used for creating the final output
matches = matches.reset_index(names=['ID1','ID2'])
matches

,ID1,ID2,condensed_name,company_name,address_1,state,email,url
0,1466,1465,1.0,1.0,0.0,1,0.0,0.708333
1,1922,1920,1.0,1.0,0.0,1,0.0,0.640000
2,2487,2486,1.0,1.0,0.0,1,0.0,0.680000
3,2574,1206,1.0,1.0,0.0,1,0.0,0.551724
4,2670,2669,1.0,1.0,0.0,1,0.0,0.564103
...,...,...,...,...,...,...,...,...
2839,97945,97944,1.0,1.0,1.0,1,0.0,0.000000
2840,98003,98002,1.0,1.0,1.0,1,1.0,0.000000
2841,98014,98013,1.0,1.0,1.0,1,0.0,0.000000
2842,98084,98083,1.0,1.0,1.0,1,0.0,0.000000


In [11]:
#Returns the values of a specific column in a Dataframe
def getColumnValues(df, column_name: str) -> list[str]:
    return df[column_name]

In [12]:
#Retrieves records from a DataFrame based on the values in a specified ID column.
def getRecordsByID(df, ID_Column_header: str) -> list[str]:
    column_values = getColumnValues(df, ID_Column_header)
    record_list = [record_data.columns.values.tolist()]
    for value in column_values:
        row = record_data[record_data['ID'] == value]
        record_list.append(row.values.tolist())
    return record_list


In [13]:
#Gets a list of record information from the record_data dataframe of the records that were in the matches dataframe
ID_list1 = getRecordsByID(matches,"ID1")
ID_list2 = getRecordsByID(matches,"ID2")

In [15]:
#Converts record data from a list of records and converts it into a Dataframe
def flattenRecords(record_list) -> pd.DataFrame:
    flattened_data = [item for sublist in record_list[1:] for item in sublist]
    df=pd.DataFrame(flattened_data, columns= ['ID', 'firm_id', 'company_name', 'condensed_name', 
                                              'main', 'legal', 'address_1', 'address_2', 'city', 
                                              'state', 'zip', 'zip_4', 'phone', 'email', 'url'])
    df.drop(['condensed_name','main','legal','zip','zip_4'],inplace=True,axis=1)
    return df

In [19]:
#Uses the above function to convert the list of values from record_data using the IDs from the matches dataframe
df1 = flattenRecords(ID_list1)
df2 = flattenRecords(ID_list2)

In [14]:
df1

,ID,firm_id,company_name,address_1,address_2,city,state,phone,email,url
0,1466,1063.0,Superamerica,7767 Elm Creek Blvd N Ste 100,NaN,Maple Grove,MN,6128274681,NaN,http://www.jbhudson.com
1,1922,1489.0,St. Paul Hotel,701 Elm Ave W,NaN,Waseca,MN,3202593198,newtub01@gmail.com,http://www.doranking.com
2,2487,2228.0,A 1 Handyman,8098 Excelsior Blvd,NaN,Hopkins,MN,6128240410,buchkosky@popp.net,http://www.iphouse.com
3,2574,2369.0,"J.D. Fingerman Ent., Inc.",9110 Meadowview Rd,NaN,Bloomington,MN,6517768580,jesgardner40@hotmail.com,http://www.oakdalerental.com
4,2670,2705.0,"Movers, Inc.",PO Box 335,19300 Linden Dr,Rogers,MN,NaN,joe@kissalaska.com,https://www.millsauto.com
...,...,...,...,...,...,...,...,...,...,...
2839,97945,96154105.0,"Frank T. Mabley, Attorney at Law",503 Saint Paul St,NaN,Saint Peter,MN,5074658418,marketing@aha-mn.com,NaN
2840,98003,96154180.0,"Hewitt Stucco, Inc.",PO Box 246,NaN,St Peter,MN,5073548188,info@modeencompany.com,NaN
2841,98014,96154203.0,Pro-Fit Upholstery,38573 Timber Trl,NaN,Saint Peter,MN,5073594195,NaN,NaN
2842,98084,96154325.0,Shimota Furniture Service,37491 County Road 15,NaN,St Peter,MN,5073599371,craig.schlegel@yahoo.com,NaN


In [20]:
df2

,ID,firm_id,company_name,address_1,address_2,city,state,phone,email,url
0,1465,1063.0,Speedway LLC,1600 E. Madison Ave.,NaN,Mankato,MN,6127245426,Highway10mobil@att.net,https://www.facebook.com/metcalfmoving?ref=stream
1,1920,1489.0,"350 Market Street, Inc.",957 Rice St,NaN,Saint Paul,MN,5073879254,blainbrookbowl@comcast.net,http://northgatehomes.com/
2,2486,2225.0,Film & Video Service,325 Blake Rd. N.,NaN,Hopkins,MN,9528944285,wci@wollakconstruction.com,NaN
3,1206,NaN,NaN,2695 Brookview Dr E,NaN,Maplewood,MN,9529206229,NaN,http://www.libertycarton.com
4,2669,2687.0,Menards Cashway Lumber,500 Cardigan Rd,NaN,Saint Paul,MN,9528301133,info@counselwealthmanagement.com,http://www.tcwreckersales.com
...,...,...,...,...,...,...,...,...,...,...
2839,97944,96154105.0,Greenstein Mabley & Wall LLC,503 Saint Paul St,NaN,St Peter,MN,5074633443,sales@alertdistributing.com,NaN
2840,98002,96154179.0,Foremost Technologies,35537 County Road 15,NaN,Saint Peter,MN,5072333252,gregh@modeencompany.com,NaN
2841,98013,96154202.0,Kranz Brothers Truck & Excavtg,38573 Timber Trl,NaN,St Peter,MN,5073599137,meg.g@thegenevasuites.com,NaN
2842,98083,96154324.0,Storlie - Pladson Chiropractic,41097 436th St,NaN,Saint Peter,MN,5073541212,mshepard@walser.com,NaN
